In [9]:
from openai import OpenAI

In [10]:
# Connection parameters (only has to be executed once during a session)

# Note: `localhost` is only valid as long as you are logged in on madagaskarweihe. 
# Otherwise, you will need to perform port forwarding on your machine using `ssh -fNL 8006:127.0.0.1:8006 ashousaa@madagaskarweihe`.
# This might require an ssh key to be uploaded to madagaskarweihe first.
VLLM_API_ENDPOINT = 'http://localhost:8006/v1' 
VLLM_KEY = 's7Vtzeyq3kfhVkPhlWdL95pPRBq36KDP1d5bBj54BqQ'
MODEL = "default"

In [11]:
# Connection to LLM server (only has to be executed once during a session)
client = OpenAI(api_key=VLLM_KEY,
                base_url=VLLM_API_ENDPOINT)

In [12]:

# Split questions by ?
import docx
import csv

def split_questions_by_marker(doc_path, marker="?"):
    doc = docx.Document(doc_path)
    full_text = ""
    for paragraph in doc.paragraphs:
        full_text += paragraph.text + "\n" # Add newline to separate paragraphs

    # Split on "?" but keep it using custom logic
    raw_parts = full_text.split(marker)
    questions = []

    for part in raw_parts:
        cleaned = part.strip() # Remove leading/trailing whitespace
        if cleaned:
            questions.append(cleaned + marker)  # Add ? back

    return questions

# Example usage
doc_path = "Manual Ques (after modification) .docx"
questions = split_questions_by_marker(doc_path)

# # Save questions with ids as a csv file
# def save_chunks_to_csv(questions, output_file="original_questions.csv"):
#     with open(output_file, mode="w", encoding="utf-8", newline="") as file:
#         writer = csv.writer(file)
#         writer.writerow(["question_id", "question_text"])  # Write header

#         for i, question in enumerate(questions):
#             question_id = f"question_{i + 1}"
#             writer.writerow([question_id, question])
#             writer.writerow([]) # Empty row to create a line break

# # Call the save_chunks_to_csv function with the values from the chunk_dict
# save_chunks_to_csv(questions=questions, output_file="original_questions.csv")

In [13]:
# # Prompt LLM To Paraphrase Questions
# with open("paraphrased_questions.txt", "w", encoding="utf-8") as f:
#     for i, question in enumerate(questions):

#         messages = [
#             {"role": "system", "content": "You are a helpful assistant that paraphrases questions. Do NOT repeat the question exactly. Do NOT add new information."},
#             {"role": "user", "content": f"Paraphrase the following question: {question}"}
#         ]
        
#         # Send prompts and wait for answer
#         response = client.chat.completions.create(
#             model=MODEL,
#             messages=messages,
#             seed=42,#  An initial value that initializes the random number generator in a generative model. For reproducibility you can set it to a fixed value.
#             temperature=0.7, # A parameter that controls the randomness and creativity of a generative model's output.
#         )

#         # Extract and clean the LLM's response
#         paraphrased = response.choices[0].message.content.strip() 

#         # Save paraphrased question
#         f.write(f"Q{i+1} Paraphrased: {paraphrased}\n\n")

# Prompt LLM To Paraphrase Questions
# Prompt LLM To Paraphrase Questions
with open("modified_paraphrased_questions.txt", "w", encoding="utf-8") as f:
    for i, question in enumerate(questions):

        messages = [
            {"role": "system", "content": "You are a helpful assistant that paraphrases questions. Do NOT repeat the question exactly. Do NOT add new information."},
            {"role": "user", "content": f"Paraphrase the following question: {question}"}
        ]
        
        # Send prompts and wait for answer
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            seed=42,
            temperature=0.7,
        )

        # Extract and clean the LLM's response
        content = response.choices[0].message.content.strip()

        # Remove introductory lines like "Here are a few ways..."
        lines = content.splitlines()
        for line in lines:
            line = line.strip()
            if not line:
                continue
            # Remove bullets or prefixes and format properly
            if line.startswith(("*", "-", "•")):
                line = line[1:].strip()
                f.write(f"Q{i+1} Paraphrased: {line}\n")
            elif not line.lower().startswith("here are"):
                f.write(f"Q{i+1} Paraphrased: {line}\n")
        f.write("\n")
